In [40]:
import praw

REDDIT_CLIENT_ID = 'ijE6EIkzU1bQGXmRLn-7RQ'
REDDIT_SECRET = 'ex7i5dJu9Y2PBldLKhG7sw9BW-5Wiw'
REDDIT_USER_AGENT = 'StockPredictor v1.0 by /u/Similar_Earth1995'  

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

In [41]:
subreddit_name = 'stocks' 
subreddit = reddit.subreddit(subreddit_name)

posts = subreddit.top(time_filter='month', limit=5000)  

data = []
for post in posts:
    data.append({
        'title': post.title,
        'selftext': post.selftext,
        'upvotes': post.score,
        'comments': post.num_comments,
        'created_utc': post.created_utc
    })


In [42]:
import pandas as pd

df = pd.DataFrame(data)

df.to_csv('reddit_posts_data.csv', index=False)

print("Data saved to 'reddit_posts_data.csv'")


Data saved to 'reddit_posts_data.csv'


Topic Modeling

In [45]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stop words
    return ' '.join(tokens)

# Apply cleaning
df['cleaned_title'] = df['title'].apply(clean_text)
df['cleaned_selftext'] = df['selftext'].apply(clean_text)

# Save cleaned data
df.to_csv('reddit_cleaned_posts.csv', index=False)
print("Cleaned data saved to 'reddit_cleaned_posts.csv'")


[nltk_data] Downloading package stopwords to C:\Users\Challa Sai
[nltk_data]     Chetana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Challa Sai
[nltk_data]     Chetana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Cleaned data saved to 'reddit_cleaned_posts.csv'


In [46]:
from sklearn.feature_extraction.text import CountVectorizer

df['text'] = df['cleaned_title'] + ' ' + df['cleaned_selftext']

# Use CountVectorizer to create a document-term matrix
vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words='english')
dtm = vectorizer.fit_transform(df['text'])

print("Document-Term Matrix shape:", dtm.shape)


Document-Term Matrix shape: (338, 692)


In [47]:
from sklearn.decomposition import LatentDirichletAllocation

# Define the number of topics
num_topics = 5

# Fit LDA model
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

print("LDA Model Fitted!")


LDA Model Fitted!


In [48]:
# Get feature names (words)
words = vectorizer.get_feature_names_out()

# Display topics and their top words
for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx + 1}:")
    print(" ".join([words[i] for i in topic.argsort()[-10:]]))
    print()


Topic 1:
potential share revenue ai data growth price stock company market

Topic 2:
shares know time think buy company im years like stock

Topic 3:
growth share earnings company quarter said year revenue million billion

Topic 4:
trump companies energy like sp tesla im market stocks stock

Topic 5:
including investopedia links aggregated options stocks daily fundamentals market news



In [49]:
# Get the topic distribution for each document
doc_topic_matrix = lda.transform(dtm)

# Assign the dominant topic for each document
df['dominant_topic'] = doc_topic_matrix.argmax(axis=1)

# Save the results
df.to_csv('reddit_topic_modeling.csv', index=False)
print("Topic modeling results saved to 'reddit_topic_modeling.csv'")


Topic modeling results saved to 'reddit_topic_modeling.csv'


In [50]:
import pandas as pd

# Load the data
df = pd.read_csv('reddit_topic_modeling.csv')

import numpy as np
df['stock_movement'] = np.random.choice(['up', 'down'], size=len(df))

print(df.head())


                                               title  \
0  Elon Musk’s Trump bet has paid off so well tha...   
1  If you could only buy one stock now what would...   
2  Nvidia to join Dow Jones Industrial Average, r...   
3        TSMC cannot make 2nm chips abroad now: MOEA   
4  Warren Buffett’s Berkshire Hathaway sold ~$14 ...   

                                            selftext  upvotes  comments  \
0  [https://fortune.com/2024/11/11/elon-musk-dona...     9728       763   
1  \nI have $2000 to invest. I’m stupid when it c...     1535      2433   
2  Nvidia is replacing Intel on the Dow Jones Ind...     1177       135   
3  Taiwan’s technology protection rules prohibits...     1049       250   
4  Berkshires cash on hand hits a record $325 Bil...     1047       333   

    created_utc                                      cleaned_title  \
0  1.731344e+09  elon musks trump bet paid well tesla worth res...   
1  1.732735e+09                          could buy one stock would   
2 

In [51]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['dominant_topic_encoded'] = encoder.fit_transform(df['dominant_topic'])
df['stock_movement_encoded'] = encoder.fit_transform(df['stock_movement'])

print("Encoded columns added!")


Encoded columns added!


In [52]:
from sklearn.model_selection import train_test_split

X = df[['dominant_topic_encoded', 'upvotes', 'comments']]
y = df['stock_movement_encoded']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data split completed!")


Data split completed!


In [53]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

print("Model training completed!")


Model training completed!


In [54]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


Accuracy: 0.54
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.58      0.58        36
           1       0.52      0.50      0.51        32

    accuracy                           0.54        68
   macro avg       0.54      0.54      0.54        68
weighted avg       0.54      0.54      0.54        68



In [55]:
from textblob import TextBlob

df['sentiment_score'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

print("Sentiment scores added!")


Sentiment scores added!


In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Combine title and text
df['text'] = df['cleaned_title'] + ' ' + df['cleaned_selftext']

# Vectorize the text using CountVectorizer
vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words='english')
dtm = vectorizer.fit_transform(df['text'])

# Apply LDA
num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Add topic probabilities as features
topic_probabilities = lda.transform(dtm)
for i in range(num_topics):
    df[f'topic_{i}_prob'] = topic_probabilities[:, i]

print("Topic probabilities added!")


Topic probabilities added!


In [57]:
from imblearn.over_sampling import SMOTE

# Separate features and target variable
X = df[['sentiment_score', 'topic_0_prob', 'topic_1_prob', 'topic_2_prob', 'topic_3_prob', 'topic_4_prob', 'upvotes', 'comments']]
y = df['stock_movement_encoded']  # Assuming stock_movement_encoded is the target column

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("SMOTE applied: Dataset is now balanced!")


SMOTE applied: Dataset is now balanced!


In [58]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

print("Data split into training and testing sets!")


Data split into training and testing sets!


In [59]:
from xgboost import XGBClassifier

# Initialize and train the model
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

print("Model training completed!")


C:\Users\Challa Sai Chetana\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:44:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Model training completed!


In [60]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


Accuracy: 0.45
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.37      0.43        41
           1       0.40      0.57      0.47        30

    accuracy                           0.45        71
   macro avg       0.47      0.47      0.45        71
weighted avg       0.48      0.45      0.45        71



In [62]:
import praw

# Initialize Reddit API client
REDDIT_CLIENT_ID = 'ijE6EIkzU1bQGXmRLn-7RQ'
REDDIT_SECRET = 'ex7i5dJu9Y2PBldLKhG7sw9BW-5Wiw'
REDDIT_USER_AGENT = 'StockPredictor v1.0 by /u/Similar_Earth1995'   # e.g., "StockPredictorApp v1.0 by /u/your_username"

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# Define subreddit and fetch posts
subreddit = reddit.subreddit('stocks')  # You can change this to a relevant subreddit
posts = []

# Scraping top 5000 posts from the last month
for submission in subreddit.top('month', limit=5000):
    posts.append([submission.title, submission.selftext, submission.score, submission.num_comments])

# Convert posts into a DataFrame
import pandas as pd

df = pd.DataFrame(posts, columns=['title', 'text', 'upvotes', 'comments'])

# Check the first few entries of the dataframe
df.head()


C:\Users\Challa Sai Chetana\AppData\Local\Temp\ipykernel_24712\3742111384.py:19: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in subreddit.top('month', limit=5000):


,title,text,upvotes,comments
0,Elon Musk’s Trump bet has paid off so well tha...,[https://fortune.com/2024/11/11/elon-musk-dona...,9729,763
1,If you could only buy one stock now what would...,\nI have $2000 to invest. I’m stupid when it c...,1540,2435
2,"Nvidia to join Dow Jones Industrial Average, r...",Nvidia is replacing Intel on the Dow Jones Ind...,1179,135
3,TSMC cannot make 2nm chips abroad now: MOEA,Taiwan’s technology protection rules prohibits...,1048,250
4,Warren Buffett’s Berkshire Hathaway sold ~$14 ...,Berkshires cash on hand hits a record $325 Bil...,1045,333


In [63]:
# Clean text data: remove any non-alphabetical characters and lowercase everything
import re

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['text_cleaned'] = df['text'].apply(clean_text)


In [64]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply VADER to get sentiment scores
df['vader_sentiment_score'] = df['text_cleaned'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Assign labels based on sentiment
df['sentiment_label'] = df['vader_sentiment_score'].apply(lambda x: 1 if x > 0 else 0)

# Check the first few entries
df[['text_cleaned', 'vader_sentiment_score', 'sentiment_label']].head()


,text_cleaned,vader_sentiment_score,sentiment_label
0,httpsfortunecomelonmuskdonaldtrumpelectiontesl...,0.8429,1
1,\ni have to invest im stupid when it comes to...,0.4040,1
2,nvidia is replacing intel on the dow jones ind...,0.8683,1
3,taiwans technology protection rules prohibits ...,0.5118,1
4,berkshires cash on hand hits a record billion...,0.6808,1


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the cleaned text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X_tfidf = vectorizer.fit_transform(df['text_cleaned'])

# Convert to DataFrame for ease of use
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())


In [66]:
from imblearn.over_sampling import SMOTE

# Define target variable (sentiment label)
y = df['sentiment_label']

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf_df, y)

# Check the class distribution after SMOTE
print(pd.Series(y_resampled).value_counts())


sentiment_label
1    285
0    285
Name: count, dtype: int64


In [69]:
pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------------ --- 1.3/1.4 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [70]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train a LightGBM model
model_lgb = lgb.LGBMClassifier(random_state=42, is_unbalance=True)
model_lgb.fit(X_train, y_train)

# Evaluate the model
y_pred_lgb = model_lgb.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("Accuracy (LightGBM):", accuracy_score(y_test, y_pred_lgb))
print("Classification Report (LightGBM):")
print(classification_report(y_test, y_pred_lgb))


[LightGBM] [Info] Number of positive: 224, number of negative: 232
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7700
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 505
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491228 -> initscore=-0.035091
[LightGBM] [Info] Start training from score -0.035091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [72]:
from sklearn.model_selection import cross_val_score

# Cross-validation to check consistency of the best model
cv_scores = cross_val_score(best_model, X_resampled, y_resampled, cv=5, scoring='f1_weighted')
print(f"Cross-Validation F1 Score: {cv_scores.mean():.2f}")


[LightGBM] [Info] Number of positive: 228, number of negative: 228
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030595 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8068
[LightGBM] [Info] Number of data points in the train set: 456, number of used features: 516
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

Light bgm model gave more accuracy compared to xgboost and randomforest which is 92%